In [1]:
from source.regency import Regency
from source.run import rungame
from source.mapping import Mapping
from source.DQNAgent import DQNAgent
from IPython.display import clear_output
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
%matplotlib notebook

In [3]:
X = 1
my_game = rungame('TestGame2', train=True, GameLength=X, IntDC = 15)
Game = my_game.Game
for a in range(X):
    Game.random_events()
    Game.collect_regency_points()
    Game.domain_initiative()
    Game.collect_gold_bars()
    Game.maintenance_costs(Update=True)
    Game.take_domain_actions()
    Game.take_domain_actions()
    Game.take_domain_actions()
Game.War

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.cast instead.
Time to Finish the Rounditiative  0
Time to Finish the Rounditiative  0
Time to Finish the Rounditiative  0
   Regent                                          Full Name  Final Score  First Score  Score
64     Fh                          Fhieleraenne (Tuarhievel)          220           99    121
87     LF                           Laela Flaertes (Tuornen)          162           75     87
59     TD  Thuriene Donalls (Northern Imperial Temple of ...          189          113     76
45     Go                                         The Gorgon          386          321     65
7      Sp                                         The Spider           71            8     63
44     AB                          Aeric Boeruine (Boeruine)          251          188     6

,Year,Location,Event,Notes
0,"Roelir, year 1524 HC",Marilen,Battle of Marilen,Battle of Marilen\n\nBelligerents:\nHarald Kho...


In [4]:
Game.Score = Game.Score.sort_values('Score', ascending=False)
Game.Score.head()

,Regent,Full Name,Final Score,First Score,Score
64,Fh,Fhieleraenne (Tuarhievel),220,99,121
87,LF,Laela Flaertes (Tuornen),162,75,87
59,TD,Thuriene Donalls (Northern Imperial Temple of ...,189,113,76
45,Go,The Gorgon,386,321,65
7,Sp,The Spider,71,8,63


In [5]:
len(Game.errors)

0

In [6]:
import numpy as np
import pandas as pd
self = Game

Regent, decision, actor, Type, state, capital, high_pop, low_pop, friend, enemy, rando, enemy_capital = Game.errors[0]

IndexError: list index out of range

In [ ]:
np.argmax(decision)